In [1]:
from pathlib import Path
from pydub import AudioSegment
from utils import read_rttm
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "2"
idx = 0
min_length_seconds = 0.1
indices = [file.with_suffix("").name for file in Path("data/audio_16khz_mono_wav/").glob("*.wav")]
for idx in indices:
    audio_path = Path(f"data/audio_16khz_mono_wav/{idx}.wav")
    diarization_path = Path(f"data/diarization/{idx}.rttm")
    diarization_df = read_rttm(diarization_path)
    diarization_df["duration"] = diarization_df.end - diarization_df.start


    tempdir = Path("./temp_files")
    tempdir.mkdir(exist_ok=True)

    audio = AudioSegment.from_wav(audio_path)
    filenames = []
    for i, row in diarization_df.iterrows():
        filename = audio_path.with_suffix("").name + f"__{row['start']}__{row['end']}.wav"
        start = int(1000*row["start"])
        end = int(1000*row["end"])
        outpath = str(tempdir/filename)
        filenames.append(outpath)
        audio[start:end].export(outpath, format="wav")
    diarization_df["path"] = filenames


    from utils import process_nemo, process_whisper

    files = diarization_df.path.tolist()
    # diarization_df["nemo"] = process_nemo(files)
    diarization_df["whisper"] = process_whisper(files, lang="slovenian")
    for i in tempdir.glob("*.wav"):
        i.unlink()
    tempdir.rmdir()
    diarization_df.to_csv(f"data/asr/{idx}_diarization_whisper.csv", index=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor 